##### https://www.analyticsvidhya.com/blog/2020/08/analysing-streaming-tweets-with-python-and-postgresql/#h2_9

In [ ]:
# Twitter API authentication

import tweepy
import config
import time



In [ ]:
class MyStream(tweepy.Stream):
    
    
    def __init__(self, time_limit=300):
        
        self.start_time = time.time()
        self.limit = time_limit
        
        consumer_key = config.consumer_key
        consumer_secret = config.consumer_secret
        access_token = config.access_token
        access_token_secret = config.access_token_secret
        
        
        
        
        
        
        super().__init__(consumer_key,consumer_secret,access_token,access_token_secret)
    
        #self.file=open("tweetvidhya.csv","w")
    
    
    def on_connect(self):
        print("Connected to Twitter API.")
    

    
    def on_status(self, status):
        
        #print("Status")
        print(status.favorite_count)
        print(status.retweet_count)
        
        #with open("out.csv", "a", encoding='utf-8') as f:
        #    f.write("%s,%s,%s,%s\n" % (status.created_at, status.user.screen_name, status.text, status.retweeted))

        
        
        
        # Tweet ID
        
        
        created_at = status.created_at
        
        tweet_id = status.id
        
        tweet_id_str = status.id_str
        
        
        retweet_count = status.retweet_count
        likes = status.favorite_count
        
        # User ID
        user_id = status.user.id
        # Username
        user_name = status.user.name
        
        user_id_str = status.user.id_str
        
        
        screen_name = status.user.screen_name
        
        location = status.user.location
        
        verified = status.user.verified
        
        url = status.user.url
        
        tweet_url = None #status.url
        
        followers = status.user.followers_count
        
        following = str(status.user.following)
        
        user_creation_date = None
        
        
        coord_lat = None
        coord_long = None
        sentiment = None
        
        # Tweet
        if status.truncated == True:
            tweet = status.extended_tweet['full_text']
            hashtags = status.extended_tweet['entities']['hashtags']
        else:
            tweet = status.text
            hashtags = status.entities['hashtags']
        
        # Read hastags
        hashtags = read_hashtags(hashtags)            
        
        # Retweet count
        retweet_count = status.retweet_count
        # Language
        lang = status.lang
        
        
        # If tweet is not a retweet and tweet is in English
        if not hasattr(status, "retweeted_status") and lang=="en":
            # Connect to database
            dbConnect(user_id, user_id_str, user_name,screen_name, location, verified,url, followers, following, user_creation_date, tweet_id, tweet_url, created_at, tweet,retweet_count,likes,coord_lat, coord_long,lang,sentiment,hashtags)
            #pass
        if (time.time() - self.start_time) > self.limit:
            print(time.time(), self.start_time, self.limit)
            return False
            
    def on_error(self, status_code):
        if status_code == 420:
            # Returning False in on_data disconnects the stream
            return False

In [ ]:
 # Extract hashtags
def read_hashtags(tag_list):
    hashtags = []
    for tag in tag_list:
        hashtags.append(tag['text'])
    return hashtags

In [ ]:
import psycopg2
import config


# Insert Tweet data into database
def dbConnect(user_id,user_id_str,user_name,screen_name, location, verified,url, followers,following, user_creation_date, tweet_id, tweet_url, created_at, tweet,retweet_count,likes,coord_lat, coord_long,lang,sentiment,hashtags):
    
    # Connection to database server
    #conn = psycopg2.connect(host="localhost",database="TwitterDB",port=5432,user='postgres',password=config.db_password)
    conn = psycopg2.connect(host=config.PGHOST,database="postgres",port=5432,user='postgres',password=config.pgpassword)
    
    cur = conn.cursor()

    # insert user information
    command = '''INSERT INTO TwitterUser (user_id, user_id_str,user_name,screen_name, location, verified,url, followers,following,user_creation_date) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) ON CONFLICT (User_ID) DO NOTHING'''
   
    cur.execute(command,(user_id,user_id_str,user_name,screen_name, location, verified,url, followers,following,user_creation_date))

    # insert tweet information
    command = '''INSERT INTO TwitterTweet (tweet_id,tweet_url, created_at, tweet, retweet_count, likes, coord_lat,coord_long, language, sentiment,user_id) VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);'''
    
    cur.execute(command,(tweet_id,tweet_url, created_at, tweet,retweet_count, likes, coord_lat,coord_long, lang, sentiment,user_id))
    
    #roar = retweet_count
    
    
    # insert entity information
    for i in range(len(hashtags)):
        hashtag = hashtags[i]
        command = '''INSERT INTO TwitterEntity (tweet_id, hashtag) VALUES (%s,%s);'''
        cur.execute(command,(tweet_id, hashtag))
    
    # Commit changes
    conn.commit()
    
    # Disconnect
    cur.close()
    conn.close()

In [ ]:
# Streaming tweets


myStream = MyStream()
#myStream = tweepy.Stream()#auth=api.auth, listener=myStreamListener,
#                        tweet_mode="extended")

                     


In [ ]:
myStream.filter(track=['Pfizer/BioNTech','Sinopharm', 'moderna','pfizer','Sinovac','Oxford/AstraZeneca','Covaxin','Sputnik V'], languages = ["en"], threaded=True)
